In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker


In [2]:
years = list(range(2001, 2022))
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=FreeThrowsPercent&agg=PerGame"

In [3]:
for year in years:
    with open("freethrows_percentage_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [5]:
dfs = []
for year in years:
    with open("freethrows_percentage_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    percentage_data = pd.read_html(str(player_table))[0]
    percentage_data["Year"] = year
    dfs.append(percentage_data)

In [6]:
percentage_data = pd.concat(dfs)
percentage_data

,Rank,Player,Team,GamesG,AttemptedAtt.,Made,% Made%,Year
0,1,"CHIKALKIN, SERGEICHIKALKIN, S.",TREBenetton BasketBenetton Basket,21,2.29,2.10,91.67%,2001
1,2,"FEMERLING, PATRICKFEMERLING, P.",OLYOlympiacosOlympiacos,14,2.21,2.00,90.32%,2001
2,3,"SMODIS, MATJAZSMODIS, M.",VIRKinder Virtus BolognaKinder Virtus Bologna,22,2.23,2.00,89.80%,2001
3,4,"DJORDJEVIC, SASADJORDJEVIC, S.",RMDReal MadridReal Madrid,17,4.00,3.59,89.71%,2001
4,5,"RIGAUDEAU, ANTOINERIGAUDEAU, A.",VIRKinder Virtus BolognaKinder Virtus Bologna,21,1.81,1.62,89.47%,2001
...,...,...,...,...,...,...,...,...
45,46,"CANAAN, ISAIAHCANAAN, I.",UNKUNICS KazanUNICS Kazan,24,2.04,1.67,81.63%,2021
46,47,"BALDWIN IV, WADEBALDWIN, W.",BKNBitci Baskonia Vitoria-GasteizBitci Baskoni...,30,3.20,2.60,81.25%,2021
47,48,"OSETKOWSKI, DYLANOSETKOWSKI, D.",ASVLDLC ASVEL VilleurbanneLDLC ASVEL Villeurbanne,26,1.42,1.15,81.08%,2021
48,49,"GIEDRAITIS, ROKASGIEDRAITIS, R.",BKNBitci Baskonia Vitoria-GasteizBitci Baskoni...,29,2.14,1.72,80.65%,2021


In [7]:
engine = create_engine('sqlite:///freethrows_percentage_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
percentage_data.to_sql(
    name = "freethrows_percentage_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [ ]:
freethrows_percentage_leaders_select = session.execute('select * from "freethrows_percentage_leaders"')
freethrows_percentage_leaders_select.all()